# Feature Engineering

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy as sp
import textblob
import sklearn
from textblob import TextBlob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

print(f"System version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {sp.__version__}")
print(f"textblob version: {textblob.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")

## Extracción

En esta sección, extraemos los datos de los archivos steam_games, user_items y user_reviews que estan en formato parquet.

In [ ]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(f'../dataset/{name}.parquet', engine='pyarrow')
    return dataframes


parquet_files = ['steam_games', 'user_items', 'user_reviews']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_steam_games = dataframes['steam_games']
df_user_items = dataframes['user_items']
df_user_reviews = dataframes['user_reviews']

Vamos a añadir una columna ‘sentiment_analysis’ al dataset ‘user_reviews’ usando NLP para analizar el sentimiento de las reseñas de los juegos. Esto nos permitirá entender las opiniones de los usuarios. Las reseñas se calificarán de la siguiente manera:

0: Negativa (insatisfacción, disgusto, decepción)
1: Neutral (indiferencia, objetividad, sin emoción)
2: Positiva (satisfacción, gusto, admiración)

Crearemos una función **`analisis_sentimiento`** usando TextBlob para analizar el sentimiento de las reseñas de los juegos. Esta función se basará en la polaridad, que varía entre -1 y 1, para determinar si una reseña es negativa, neutra o positiva.

In [ ]:
def analisis_sentimiento(review):
    # Si la reseña está ausente, retorna 1 (neutral)
    if pd.isnull(review):
        return 1

    # Calcula la polaridad de la reseña usando TextBlob
    polarity = TextBlob(review).sentiment.polarity

    # Retorna 0 (malo) si la polaridad es menor que 0, 2 (positivo) si la polaridad es mayor que 0, y 1 (neutral) en caso contrario
    if polarity < 0:
        return 0
    elif polarity > 0:
        return 2
    else:
        return 1

- Aplicamos la función a la columna review.

In [ ]:
df_user_reviews.sample(5)

In [ ]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(analisis_sentimiento)
df_user_reviews[['review','sentiment_analysis']].sample(5)

## Creación de Conjuntos de Datos para los Endpoints de la API

Nuestro propósito en esta sección es establecer varios conjuntos de datos, actuando como bases de datos pseudo, para las funciones de los endpoints de la API. Esto nos permitirá recuperar los datos requeridos de manera rápida y eficaz, sin la necesidad de cargar toda la información, optimizando así el rendimiento de la API.

## Creación de la Pseudo Base de Datos 1
####    (Endpoints de la API def PlayTimeGenre( genero : str ): y def UserForGenre( genero : str ):)
Para formar un único conjunto de datos que sirva como pseudo base de datos para los endpoints, es necesario fusionar df_steam_games y df_user_items. De esta forma, consolidamos toda la información requerida en un solo lugar. Las columnas necesarias son: item_id, genres, release_year de df_steam_games y item_id, user_id, playtime_forever de df_user_items.

1. Seleccionamos únicamente las columnas requeridas:
```python
steam_games_columns = ['item_id','genres','release_year']
user_items_columns = ['item_id','user_id', 'playtime_forever']
```
2. Creamos subconjuntos de los DataFrames con solo las columnas necesarias:
```python
df_games_subset = df_steam_games[steam_games_columns]
df_items_subset = df_user_items[user_items_columns]
```

In [ ]:
steam_games_columns = ['item_id','genres','release_year']
user_items_columns = ['item_id','user_id', 'playtime_forever']

df_games_subset = df_steam_games[steam_games_columns]
df_items_subset = df_user_items[user_items_columns]



df_endpoints1_2 = pd.merge(df_games_subset, df_items_subset, on='item_id')
df_endpoints1_2.head(5)

In [ ]:
# Seleccionamos los 10 géneros mas frecuentes
top_10_popular_genres = ['Action', 'Adventure', 'Indie', 'Strategy', 'RPG', 'Simulation', 'Casual', 'Massively Multiplayer', 'Racing', 'Sports']

# Filtramos por las condiciones establecidas
df_endpoints1_2 = df_endpoints1_2[(df_endpoints1_2['release_year'] != 'unknown') & (df_endpoints1_2['playtime_forever'] > 0)].reset_index(drop=True)
df_endpoints1_2.head()